### import packages

In [1]:
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
import lightgbm as lgb

pd.set_option('display.max_columns', None)

In [2]:
def data_process(prop):
    prop.rename(columns={
        'parcelid': 'parcelid',  # Unique identifier of parcels
        'airconditioningtypeid': 'cooling_id',  # type of cooling system (if any), 1~13
        'architecturalstyletypeid': 'architecture_style_id',  # Architectural style of the home, 1~27
        'basementsqft': 'basement_sqft',  # Size of the basement
        'bathroomcnt': 'bathroom_cnt',  # Number of bathrooms (including fractional bathrooms)
        'bedroomcnt': 'bedroom_cnt',  # Number of bedrooms
        'buildingclasstypeid': 'framing_id',  # The building framing type, 1~5
        'buildingqualitytypeid': 'quality_id',  # building condition from best (lowest) to worst (highest)
        'calculatedbathnbr': 'bathroom_cnt_calc',  # Same meaning as 'bathroom_cnt'?
        'decktypeid': 'deck_id',  # Type of deck (if any)
        'finishedfloor1squarefeet': 'floor1_sqft',  # Size of finished living area on first floor
        'calculatedfinishedsquarefeet': 'finished_area_sqft_calc',  # calculated total finished living area
        'finishedsquarefeet12': 'finished_area_sqft',  # Same meaning as 'finished_area_sqft_calc'?
        'finishedsquarefeet13': 'perimeter_area',  # Perimeter living area
        'finishedsquarefeet15': 'total_area',  # Total area
        'finishedsquarefeet50': 'floor1_sqft_unk',  # Same meaning as 'floor1_sqft'?
        'finishedsquarefeet6': 'base_total_area',  # Base unfinished and finished area
        'fips': 'fips',  # Federal Information Processing Standard code
        'fireplacecnt': 'fireplace_cnt',  # Number of fireplaces in the home (if any)
        'fullbathcnt': 'bathroom_full_cnt',  # Number of full bathrooms
        'garagecarcnt': 'garage_cnt',  # Total number of garages
        'garagetotalsqft': 'garage_sqft',  # Total size of the garages
        'hashottuborspa': 'spa_flag',  # Whether the home has a hot tub or spa
        'heatingorsystemtypeid': 'heating_id',  # type of heating system, 1~25
        'latitude': 'latitude',  # latitude of the middle of the parcel multiplied by 1e6
        'longitude': 'longitude',  # longitude of the middle of the parcel multiplied by 1e6
        'lotsizesquarefeet': 'lot_sqft',  # Area of the lot in sqft
        'poolcnt': 'pool_cnt', # Number of pools in the lot (if any)
        'poolsizesum': 'pool_total_size',  # Total size of the pools
        'pooltypeid10': 'pool_unk_1',
        'pooltypeid2': 'pool_unk_2',
        'pooltypeid7': 'pool_unk_3',
        'propertycountylandusecode': 'county_landuse_code',
        'propertylandusetypeid': 'landuse_type_id' ,  # Type of land use the property is zoned for, 25 categories
        'propertyzoningdesc': 'zoning_description',  # Allowed land uses (zoning) for that property
        'rawcensustractandblock': 'census_1',
        'regionidcity': 'city_id',  # City in which the property is located (if any)
        'regionidcounty': 'county_id',  # County in which the property is located
        'regionidneighborhood': 'neighborhood_id',  # Neighborhood in which the property is located
        'regionidzip': 'region_zip',
        'roomcnt': 'room_cnt',  # Total number of rooms in the principal residence
        'storytypeid': 'story_id',  # Type of floors in a multi-story house, 1~35
        'threequarterbathnbr': 'bathroom_small_cnt',  # Number of 3/4 bathrooms
        'typeconstructiontypeid': 'construction_id',  # Type of construction material, 1~18
        'unitcnt': 'unit_cnt',  # Number of units the structure is built into (2=duplex, 3=triplex, etc)
        'yardbuildingsqft17': 'patio_sqft',  # Patio in yard
        'yardbuildingsqft26': 'storage_sqft',  # Storage shed/building in yard
        'yearbuilt': 'year_built',  # The year the principal residence was built
        'numberofstories': 'story_cnt',  # Number of stories or levels the home has
        'fireplaceflag': 'fireplace_flag',  # Whether the home has a fireplace
        'structuretaxvaluedollarcnt': 'tax_structure',
        'taxvaluedollarcnt': 'tax_parcel',
        'assessmentyear': 'tax_year',  # The year of the property tax assessment (2015 for 2016 data)
        'landtaxvaluedollarcnt': 'tax_land',
        'taxamount': 'tax_property',
        'taxdelinquencyflag': 'tax_overdue_flag',  # Property taxes are past due as of 2015
        'taxdelinquencyyear': 'tax_overdue_year',  # Year for which the unpaid propert taxes were due
        'censustractandblock': 'census_2'
    }, inplace=True)   
    
    def convert_true_to_float(df, col):
        df.loc[df[col] == 'true', col] = '1'
        df.loc[df[col] == 'Y', col] = '1'
        df[col] = df[col].astype(float)
    
    for col in ['spa_flag','fireplace_flag','tax_overdue_flag']:
        convert_true_to_float(prop, col)
    
    return prop

In [3]:
%%time
#properties of houses, need to predict each house logerror 
properties_2016_df = pd.read_csv('/glade/scratch/wmingch/ML_project/Zillow_prize/properties_2016.csv') 
properties_2017_df = pd.read_csv('/glade/scratch/wmingch/ML_project/Zillow_prize/properties_2017.csv')

<string>:2: DtypeWarning: Columns (22,32,34,49,55) have mixed types. Specify dtype option on import or set low_memory=False.
<string>:2: DtypeWarning: Columns (49) have mixed types. Specify dtype option on import or set low_memory=False.


CPU times: user 14.2 s, sys: 2.01 s, total: 16.2 s
Wall time: 16.4 s


In [4]:
print('properties_2016_df shape: {}'.format(properties_2016_df.shape))
print('properties_2017_df shape: {}'.format(properties_2017_df.shape))

properties_2016_df shape: (2985217, 58)
properties_2017_df shape: (2985217, 58)


In [5]:
properties_2016_df.head()

,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,finishedfloor1squarefeet,calculatedfinishedsquarefeet,finishedsquarefeet12,finishedsquarefeet13,finishedsquarefeet15,finishedsquarefeet50,finishedsquarefeet6,fips,fireplacecnt,fullbathcnt,garagecarcnt,garagetotalsqft,hashottuborspa,heatingorsystemtypeid,latitude,longitude,lotsizesquarefeet,poolcnt,poolsizesum,pooltypeid10,pooltypeid2,pooltypeid7,propertycountylandusecode,propertylandusetypeid,propertyzoningdesc,rawcensustractandblock,regionidcity,regionidcounty,regionidneighborhood,regionidzip,roomcnt,storytypeid,threequarterbathnbr,typeconstructiontypeid,unitcnt,yardbuildingsqft17,yardbuildingsqft26,yearbuilt,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,10754147,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6037.0,NaN,NaN,NaN,NaN,NaN,NaN,34144442.0,-118654084.0,85768.0,NaN,NaN,NaN,NaN,NaN,010D,269.0,NaN,6.037800e+07,37688.0,3101.0,NaN,96337.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,2015.0,9.0,NaN,NaN,NaN,NaN
1,10759547,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6037.0,NaN,NaN,NaN,NaN,NaN,NaN,34140430.0,-118625364.0,4083.0,NaN,NaN,NaN,NaN,NaN,0109,261.0,LCA11*,6.037800e+07,37688.0,3101.0,NaN,96337.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27516.0,2015.0,27516.0,NaN,NaN,NaN,NaN
2,10843547,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,73026.0,NaN,NaN,73026.0,NaN,NaN,6037.0,NaN,NaN,NaN,NaN,NaN,NaN,33989359.0,-118394633.0,63085.0,NaN,NaN,NaN,NaN,NaN,1200,47.0,LAC2,6.037703e+07,51617.0,3101.0,NaN,96095.0,0.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,650756.0,1413387.0,2015.0,762631.0,20800.37,NaN,NaN,NaN
3,10859147,NaN,NaN,NaN,0.0,0.0,3.0,7.0,NaN,NaN,NaN,5068.0,NaN,NaN,5068.0,NaN,NaN,6037.0,NaN,NaN,NaN,NaN,NaN,NaN,34148863.0,-118437206.0,7521.0,NaN,NaN,NaN,NaN,NaN,1200,47.0,LAC2,6.037141e+07,12447.0,3101.0,27080.0,96424.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1948.0,1.0,NaN,571346.0,1156834.0,2015.0,585488.0,14557.57,NaN,NaN,NaN
4,10879947,NaN,NaN,NaN,0.0,0.0,4.0,NaN,NaN,NaN,NaN,1776.0,NaN,NaN,1776.0,NaN,NaN,6037.0,NaN,NaN,NaN,NaN,NaN,NaN,34194168.0,-118385816.0,8512.0,NaN,NaN,NaN,NaN,NaN,1210,31.0,LAM1,6.037123e+07,12447.0,3101.0,46795.0,96450.0,0.0,NaN,NaN,NaN,1.0,NaN,NaN,1947.0,NaN,NaN,193796.0,433491.0,2015.0,239695.0,5725.17,NaN,NaN,NaN


In [7]:
#rename features to make it understandable and convert true/y to 1 
properties_2016_df = data_process(properties_2016_df)
properties_2017_df = data_process(properties_2017_df)

In [10]:
properties_2017_df.head()

,parcelid,cooling_id,architecture_style_id,basement_sqft,bathroom_cnt,bedroom_cnt,framing_id,quality_id,bathroom_cnt_calc,deck_id,floor1_sqft,finished_area_sqft_calc,finished_area_sqft,perimeter_area,total_area,floor1_sqft_unk,base_total_area,fips,fireplace_cnt,bathroom_full_cnt,garage_cnt,garage_sqft,spa_flag,heating_id,latitude,longitude,lot_sqft,pool_cnt,pool_total_size,pool_unk_1,pool_unk_2,pool_unk_3,county_landuse_code,landuse_type_id,zoning_description,census_1,city_id,county_id,neighborhood_id,region_zip,room_cnt,story_id,bathroom_small_cnt,construction_id,unit_cnt,patio_sqft,storage_sqft,year_built,story_cnt,fireplace_flag,tax_structure,tax_parcel,tax_year,tax_land,tax_property,tax_overdue_flag,tax_overdue_year,census_2
0,10754147,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6037.0,NaN,NaN,NaN,NaN,NaN,NaN,34144442.0,-118654084.0,85768.0,NaN,NaN,NaN,NaN,NaN,010D,269.0,NaN,6.037800e+07,37688.0,3101.0,NaN,96337.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,2016.0,9.0,NaN,NaN,NaN,NaN
1,10759547,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6037.0,NaN,NaN,NaN,NaN,NaN,NaN,34140430.0,-118625364.0,4083.0,NaN,NaN,NaN,NaN,NaN,0109,261.0,LCA11*,6.037800e+07,37688.0,3101.0,NaN,96337.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27516.0,2015.0,27516.0,NaN,NaN,NaN,NaN
2,10843547,NaN,NaN,NaN,0.0,0.0,5.0,NaN,NaN,NaN,NaN,73026.0,NaN,NaN,73026.0,NaN,NaN,6037.0,NaN,NaN,NaN,NaN,NaN,NaN,33989359.0,-118394633.0,63085.0,NaN,NaN,NaN,NaN,NaN,1200,47.0,LAC2,6.037703e+07,51617.0,3101.0,NaN,96095.0,0.0,NaN,NaN,NaN,2.0,NaN,NaN,1959.0,1.0,NaN,660680.0,1434941.0,2016.0,774261.0,20800.37,NaN,NaN,NaN
3,10859147,NaN,NaN,NaN,0.0,0.0,3.0,6.0,NaN,NaN,NaN,5068.0,NaN,NaN,5068.0,NaN,NaN,6037.0,NaN,NaN,NaN,NaN,NaN,NaN,34148863.0,-118437206.0,7521.0,NaN,NaN,NaN,NaN,NaN,1200,47.0,LAC2,6.037141e+07,12447.0,3101.0,27080.0,96424.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1948.0,1.0,NaN,580059.0,1174475.0,2016.0,594416.0,14557.57,NaN,NaN,NaN
4,10879947,NaN,NaN,NaN,0.0,0.0,4.0,NaN,NaN,NaN,NaN,1776.0,NaN,NaN,1776.0,NaN,NaN,6037.0,NaN,NaN,NaN,NaN,NaN,NaN,34194168.0,-118385816.0,8512.0,NaN,NaN,NaN,NaN,NaN,1210,31.0,LAM1,6.037123e+07,12447.0,3101.0,46795.0,96450.0,0.0,NaN,NaN,NaN,1.0,NaN,NaN,1947.0,1.0,NaN,196751.0,440101.0,2016.0,243350.0,5725.17,NaN,NaN,NaN


In [9]:
for c in properties_2016_df.columns:
    if c not in ['parcelid']:
        print(c)
        print(properties_2016_df[c].value_counts())

cooling_id
1.0     742364
13.0     58457
5.0       8795
11.0      1818
12.0        59
9.0         19
3.0          7
Name: cooling_id, dtype: int64
architecture_style_id
7.0     5251
8.0      380
2.0      201
21.0     150
3.0       58
5.0       19
27.0       1
10.0       1
Name: architecture_style_id, dtype: int64
basement_sqft
1528.0    25
700.0     21
240.0     21
100.0     18
144.0     16
          ..
1434.0     1
1443.0     1
1448.0     1
1453.0     1
1391.0     1
Name: basement_sqft, Length: 751, dtype: int64
bathroom_cnt
2.00     1218664
3.00      629351
1.00      499330
2.50      206856
4.00      132076
0.00      116614
1.50       45885
5.00       38241
3.50       30755
4.50       19017
6.00       16252
7.00        6149
5.50        5747
8.00        4537
9.00        1329
6.50        1217
10.00        500
7.50         380
12.00        265
11.00        207
8.50         106
13.00         52
9.50          50
14.00         45
16.00         30
15.00         24
0.50          17
18.00    

96987.0    22021
96193.0    21759
97118.0    20612
97319.0    20310
96964.0    19898
           ...  
96453.0        1
96068.0        1
97096.0        1
96953.0        1
96258.0        1
Name: region_zip, Length: 405, dtype: int64
room_cnt
0.0     2308879
6.0      178357
7.0      156397
8.0      120227
5.0       99582
9.0       45241
4.0       42320
10.0      10862
3.0        6289
11.0       3021
12.0       1097
2.0         751
13.0        313
14.0        156
1.0          77
15.0         67
16.0         35
18.0         22
30.0         14
17.0          9
19.0          6
21.0          4
20.0          3
22.0          1
24.0          1
96.0          1
40.0          1
42.0          1
45.0          1
52.0          1
56.0          1
67.0          1
77.0          1
84.0          1
86.0          1
31.0          1
Name: room_cnt, dtype: int64
story_id
7.0    1624
Name: story_id, dtype: int64
bathroom_small_cnt
1.0    308959
2.0      2338
3.0       261
4.0        46
5.0        16
6.0         9
7.

In [11]:
#land use code 

def get_landuse_code_df(properties_2016_df, properties_2017_df):
    temp = properties_2016_df.groupby('county_landuse_code')['county_landuse_code'].count()
    landuse_codes = list(temp[temp >= 300].index)
    temp = properties_2017_df.groupby('county_landuse_code')['county_landuse_code'].count()
    landuse_codes += list(temp[temp >= 300].index) 
    landuse_codes = list(set(landuse_codes))
    
    df_landuse_codes = pd.DataFrame({'county_landuse_code': landuse_codes, 'county_landuse_code_id': range(len(landuse_codes))})
    return df_landuse_codes

df_landuse_codes = get_landuse_code_df(properties_2016_df, properties_2017_df)

In [12]:
#zoning des use code 

def get_zoning_desc_code_df(properties_2016_df, properties_2017_df):
    temp = properties_2016_df.groupby('zoning_description')['zoning_description'].count().sort_values(ascending=False)
    zoning_codes = list(temp[temp > 300].index)
    temp = properties_2017_df.groupby('zoning_description')['zoning_description'].count().sort_values(ascending=False)
    zoning_codes += list(temp[temp > 300].index)
    zoning_codes = list(set(zoning_codes))
    
    df_zoning_codes = pd.DataFrame({'zoning_description': zoning_codes, 'zoning_description_id':range(len(zoning_codes))})

    return df_zoning_codes

df_zoning_codes = get_zoning_desc_code_df(properties_2016_df, properties_2017_df)

In [13]:
#merge land use and zoning codes 
#for count smaller than 300 will be NaN
properties_2016_df = properties_2016_df.merge(how='left', right=df_landuse_codes, on='county_landuse_code', copy=False)
properties_2016_df = properties_2016_df.merge(how='left', right=df_zoning_codes, on='zoning_description', copy=False)
properties_2016_df = properties_2016_df.drop(['county_landuse_code','zoning_description'],axis=1)

In [14]:
properties_2017_df = properties_2017_df.merge(how='left', right=df_landuse_codes, on='county_landuse_code', copy=False)
properties_2017_df = properties_2017_df.merge(how='left', right=df_zoning_codes, on='zoning_description', copy=False)
properties_2017_df = properties_2017_df.drop(['county_landuse_code','zoning_description'],axis=1)

In [15]:
#county_id, convert to 0, 1, and 2
properties_2016_df.loc[properties_2016_df.county_id == 3101, 'county_id'] = 0
properties_2016_df.loc[properties_2016_df.county_id == 1286, 'county_id'] = 1
properties_2016_df.loc[properties_2016_df.county_id == 2061, 'county_id'] = 2

properties_2017_df.loc[properties_2017_df.county_id == 3101, 'county_id'] = 0
properties_2017_df.loc[properties_2017_df.county_id == 1286, 'county_id'] = 1
properties_2017_df.loc[properties_2017_df.county_id == 2061, 'county_id'] = 2

## Some feature engineering 

In [16]:
for prop in [properties_2016_df, properties_2017_df]:
    prop['avg_garage_size'] = prop['garage_sqft']/prop['garage_cnt']
    prop['property_tax_per_sqft'] = prop['tax_property'] / prop['finished_area_sqft_calc']
    prop['location_1'] = prop['latitude'] + prop['longitude']
    prop['location_2'] = prop['latitude'] - prop['longitude']
    prop['location_3'] = prop['latitude'] + 0.5 * prop['longitude']
    prop['location_4'] = prop['latitude'] - 0.5 * prop['longitude']
    
    prop['missing_finished_area'] = prop['finished_area_sqft'].isnull().astype(np.float32)
    prop['missing_total_area'] = prop['total_area'].isnull().astype(np.float32)
    prop.drop(['finished_area_sqft', 'total_area'], axis=1, inplace=True)
    
    prop['missing_bathroom_cnt_calc'] = prop['bathroom_cnt_calc'].isnull().astype(np.float32)
    prop.drop(['bathroom_cnt_calc'], axis=1, inplace=True)
    
    prop['derived_room_cnt'] = prop['bedroom_cnt'] + prop['bathroom_cnt']
    mask = (prop.room_cnt >= 1)
    prop.loc[mask, 'avg_area_per_room'] = prop.loc[mask, 'finished_area_sqft_calc']/prop.loc[mask, 'room_cnt']
    
    mask = (prop.derived_room_cnt >= 1)
    prop.loc[mask, 'derived_avg_area_per_room'] = prop.loc[mask, 'finished_area_sqft_calc']/prop.loc[mask, 'derived_room_cnt']

In [17]:
properties_2016_df.head()

,parcelid,cooling_id,architecture_style_id,basement_sqft,bathroom_cnt,bedroom_cnt,framing_id,quality_id,deck_id,floor1_sqft,finished_area_sqft_calc,perimeter_area,floor1_sqft_unk,base_total_area,fips,fireplace_cnt,bathroom_full_cnt,garage_cnt,garage_sqft,spa_flag,heating_id,latitude,longitude,lot_sqft,pool_cnt,pool_total_size,pool_unk_1,pool_unk_2,pool_unk_3,landuse_type_id,census_1,city_id,county_id,neighborhood_id,region_zip,room_cnt,story_id,bathroom_small_cnt,construction_id,unit_cnt,patio_sqft,storage_sqft,year_built,story_cnt,fireplace_flag,tax_structure,tax_parcel,tax_year,tax_land,tax_property,tax_overdue_flag,tax_overdue_year,census_2,county_landuse_code_id,zoning_description_id,avg_garage_size,property_tax_per_sqft,location_1,location_2,location_3,location_4,missing_finished_area,missing_total_area,missing_bathroom_cnt_calc,derived_room_cnt,avg_area_per_room,derived_avg_area_per_room
0,10754147,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6037.0,NaN,NaN,NaN,NaN,NaN,NaN,34144442.0,-118654084.0,85768.0,NaN,NaN,NaN,NaN,NaN,269.0,6.037800e+07,37688.0,0.0,NaN,96337.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,2015.0,9.0,NaN,NaN,NaN,NaN,43.0,NaN,NaN,NaN,-84509642.0,152798526.0,-25182600.0,93471484.0,1.0,1.0,1.0,0.0,NaN,NaN
1,10759547,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6037.0,NaN,NaN,NaN,NaN,NaN,NaN,34140430.0,-118625364.0,4083.0,NaN,NaN,NaN,NaN,NaN,261.0,6.037800e+07,37688.0,0.0,NaN,96337.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27516.0,2015.0,27516.0,NaN,NaN,NaN,NaN,28.0,279.0,NaN,NaN,-84484934.0,152765794.0,-25172252.0,93453112.0,1.0,1.0,1.0,0.0,NaN,NaN
2,10843547,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,73026.0,NaN,NaN,NaN,6037.0,NaN,NaN,NaN,NaN,NaN,NaN,33989359.0,-118394633.0,63085.0,NaN,NaN,NaN,NaN,NaN,47.0,6.037703e+07,51617.0,0.0,NaN,96095.0,0.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,650756.0,1413387.0,2015.0,762631.0,20800.37,NaN,NaN,NaN,25.0,411.0,NaN,0.284835,-84405274.0,152383992.0,-25207957.5,93186675.5,1.0,0.0,1.0,0.0,NaN,NaN
3,10859147,NaN,NaN,NaN,0.0,0.0,3.0,7.0,NaN,NaN,5068.0,NaN,NaN,NaN,6037.0,NaN,NaN,NaN,NaN,NaN,NaN,34148863.0,-118437206.0,7521.0,NaN,NaN,NaN,NaN,NaN,47.0,6.037141e+07,12447.0,0.0,27080.0,96424.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1948.0,1.0,NaN,571346.0,1156834.0,2015.0,585488.0,14557.57,NaN,NaN,NaN,25.0,411.0,NaN,2.872449,-84288343.0,152586069.0,-25069740.0,93367466.0,1.0,0.0,1.0,0.0,NaN,NaN
4,10879947,NaN,NaN,NaN,0.0,0.0,4.0,NaN,NaN,NaN,1776.0,NaN,NaN,NaN,6037.0,NaN,NaN,NaN,NaN,NaN,NaN,34194168.0,-118385816.0,8512.0,NaN,NaN,NaN,NaN,NaN,31.0,6.037123e+07,12447.0,0.0,46795.0,96450.0,0.0,NaN,NaN,NaN,1.0,NaN,NaN,1947.0,NaN,NaN,193796.0,433491.0,2015.0,239695.0,5725.17,NaN,NaN,NaN,48.0,194.0,NaN,3.223632,-84191648.0,152579984.0,-24998740.0,93387076.0,1.0,0.0,1.0,0.0,NaN,NaN


In [20]:
# save properties to hdf files for predictions
dir_temp = '/glade/scratch/wmingch/ML_project/Zillow_prize/temp_data/'
properties_2016_df.to_hdf(dir_temp + 'properties_2016.h5', key='properties_2016_df', mode='w')
properties_2017_df.to_hdf(dir_temp + 'properties_2017.h5', key='properties_2017_df', mode='a')

## train_2016 and train_2017 for model training 

In [21]:
def add_simple_datetime_features(df):
    dt = pd.to_datetime(df.transactiondate).dt
    df['year'] = (dt.year - 2016).astype(int)
    df['month'] = (dt.month).astype(int)
    df['quarter'] = (dt.quarter).astype(int)
    df.drop(['transactiondate'], axis=1, inplace=True)

In [22]:
%%time
target_2016_df = pd.read_csv('/glade/scratch/wmingch/ML_project/Zillow_prize/train_2016_v2.csv',parse_dates=["transactiondate"])
target_2017_df = pd.read_csv('/glade/scratch/wmingch/ML_project/Zillow_prize/train_2017.csv',parse_dates=["transactiondate"])

train_2016_df = target_2016_df.merge(how='left', right=properties_2016_df, on='parcelid')
train_2017_df = target_2017_df.merge(how='left', right=properties_2017_df, on='parcelid')

CPU times: user 2.64 s, sys: 1.14 s, total: 3.79 s
Wall time: 3.87 s


In [26]:
# add datetime features 
add_simple_datetime_features(train_2016_df)
add_simple_datetime_features(train_2017_df)

In [29]:
# concat train_2016 and train_2017
train_combined = pd.concat([train_2016_df, train_2017_df], axis=0, ignore_index=True)

In [30]:
train_combined.head()

,parcelid,logerror,cooling_id,architecture_style_id,basement_sqft,bathroom_cnt,bedroom_cnt,framing_id,quality_id,deck_id,floor1_sqft,finished_area_sqft_calc,perimeter_area,floor1_sqft_unk,base_total_area,fips,fireplace_cnt,bathroom_full_cnt,garage_cnt,garage_sqft,spa_flag,heating_id,latitude,longitude,lot_sqft,pool_cnt,pool_total_size,pool_unk_1,pool_unk_2,pool_unk_3,landuse_type_id,census_1,city_id,county_id,neighborhood_id,region_zip,room_cnt,story_id,bathroom_small_cnt,construction_id,unit_cnt,patio_sqft,storage_sqft,year_built,story_cnt,fireplace_flag,tax_structure,tax_parcel,tax_year,tax_land,tax_property,tax_overdue_flag,tax_overdue_year,census_2,county_landuse_code_id,zoning_description_id,avg_garage_size,property_tax_per_sqft,location_1,location_2,location_3,location_4,missing_finished_area,missing_total_area,missing_bathroom_cnt_calc,derived_room_cnt,avg_area_per_room,derived_avg_area_per_room,year,month,quarter
0,11016594,0.0276,1.0,NaN,NaN,2.0,3.0,NaN,4.0,NaN,NaN,1684.0,NaN,NaN,NaN,6037.0,NaN,2.0,NaN,NaN,NaN,2.0,34280990.0,-118488536.0,7528.0,NaN,NaN,NaN,NaN,NaN,261.0,6.037107e+07,12447.0,0.0,31817.0,96370.0,0.0,NaN,NaN,NaN,1.0,NaN,NaN,1959.0,NaN,NaN,122754.0,360170.0,2015.0,237416.0,6735.88,NaN,NaN,6.037107e+13,3.0,79.0,NaN,3.999929,-84207546.0,152769526.0,-24963278.0,93525258.0,0.0,1.0,0.0,5.0,NaN,336.800000,0,1,1
1,14366692,-0.1684,NaN,NaN,NaN,3.5,4.0,NaN,NaN,NaN,NaN,2263.0,NaN,NaN,NaN,6059.0,NaN,3.0,2.0,468.0,NaN,NaN,33668120.0,-117677556.0,3643.0,NaN,NaN,NaN,NaN,NaN,261.0,6.059052e+07,32380.0,1.0,NaN,96962.0,0.0,NaN,1.0,NaN,NaN,NaN,NaN,2014.0,NaN,NaN,346458.0,585529.0,2015.0,239071.0,10153.02,NaN,NaN,NaN,36.0,NaN,234.0,4.486531,-84009436.0,151345676.0,-25170658.0,92506898.0,0.0,1.0,0.0,7.5,NaN,301.733333,0,1,1
2,12098116,-0.0040,1.0,NaN,NaN,3.0,2.0,NaN,4.0,NaN,NaN,2217.0,NaN,NaN,NaN,6037.0,NaN,3.0,NaN,NaN,NaN,2.0,34136312.0,-118175032.0,11423.0,NaN,NaN,NaN,NaN,NaN,261.0,6.037464e+07,47019.0,0.0,275411.0,96293.0,0.0,NaN,NaN,NaN,1.0,NaN,NaN,1940.0,NaN,NaN,61994.0,119906.0,2015.0,57912.0,11484.48,NaN,NaN,6.037464e+13,3.0,604.0,NaN,5.180189,-84038720.0,152311344.0,-24951204.0,93223828.0,0.0,1.0,0.0,5.0,NaN,443.400000,0,1,1
3,12643413,0.0218,1.0,NaN,NaN,2.0,2.0,NaN,4.0,NaN,NaN,839.0,NaN,NaN,NaN,6037.0,NaN,2.0,NaN,NaN,NaN,2.0,33755800.0,-118309000.0,70859.0,NaN,NaN,NaN,NaN,NaN,266.0,6.037296e+07,12447.0,0.0,54300.0,96222.0,0.0,NaN,NaN,NaN,1.0,NaN,NaN,1987.0,NaN,NaN,171518.0,244880.0,2015.0,73362.0,3048.74,NaN,NaN,6.037296e+13,13.0,626.0,NaN,3.633778,-84553200.0,152064800.0,-25398700.0,92910300.0,0.0,1.0,0.0,4.0,NaN,209.750000,0,1,1
4,14432541,-0.0050,NaN,NaN,NaN,2.5,4.0,NaN,NaN,NaN,NaN,2283.0,NaN,NaN,NaN,6059.0,NaN,2.0,2.0,598.0,NaN,NaN,33485643.0,-117700234.0,6000.0,1.0,NaN,NaN,NaN,1.0,261.0,6.059042e+07,17686.0,1.0,NaN,96961.0,8.0,NaN,1.0,NaN,NaN,NaN,NaN,1981.0,2.0,NaN,169574.0,434551.0,2015.0,264977.0,5488.96,NaN,NaN,6.059042e+13,16.0,NaN,299.0,2.404275,-84214591.0,151185877.0,-25364474.0,92335760.0,0.0,1.0,0.0,6.5,285.375,351.230769,0,1,1


In [31]:
train_combined.to_hdf(dir_temp + 'train_combined.h5', key='train_combined', mode='w')